In [1]:
import sys
sys.path.append('../')

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from utils import data_loading, model
import os
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib
from matplotlib import pyplot as plt
from nilearn import datasets
from nilearn import plotting
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as corr
from sklearn.decomposition import PCA

In [15]:
def load_images_from_folder(folder_path, start=None, end=None):
    images = []
    for filename in os.listdir(folder_path)[start:end]:
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)
        img_array = np.array(img)
        # .transpose((2, 0, 1))
        images.append(img_array)
    return images

def load_subject_data(subject, index_start=None, index_end=None, return_dict=False):
    current_dir = os.getcwd()
    path = '../../data/algonauts/subj0' + str(subject)
    data_lh = np.load(path + '/training_split/training_fmri/lh_training_fmri.npy')[index_start : index_end]
    data_rh = np.load(path + '/training_split/training_fmri/rh_training_fmri.npy')[index_start : index_end]
    folder_path = path+"/training_split/training_images/"
    image_data = load_images_from_folder(folder_path, index_start, index_end)
    id_list = [subject for subject in range(len(image_data))]
    return data_lh, data_rh, image_data, id_list

In [16]:
sub1_lh, sub1_rh, image1, id1 = load_subject_data(1, 0, 10)
sub2_lh, sub2_rh, image2, id2 = load_subject_data(2, 0, 10)

In [20]:
brain1 = np.concatenate((sub1_lh, sub1_rh), axis=1)
brain2 = np.concatenate((sub2_lh, sub2_rh), axis=1)
brain = np.concatenate((brain1, brain2), axis=0)
ids = id1 + id2

In [17]:
images = image1 + image2

In [21]:
dataset = data_loading.CustomDataset(images_list=images, outputs_list=brain, transform=transforms.ToTensor(), id_list=ids)

       
Initialize CustomDataset 
--------
Number of samples:  20
Transform:  ToTensor()
PCA:  None

Data loaded
-------
Data:  20 *  ([<PIL.Image.Image image mode=RGB size=425x425 at 0x198B39BB0A0>, 0], array([-0.8617882 , -0.20318632, -0.62639767, ..., -0.41889378,
       -0.60231453, -0.67537224], dtype=float32))
Output_concat:  20 * 39548 :  [-0.8617882  -0.20318632 -0.62639767 ... -0.41889378 -0.60231453
 -0.67537224]


In [23]:
print('\nDataset made up of ', len(dataset), 'pairs of data\n--------')
print('Shape of 1st element in pair:', dataset[0][0].shape)
print('Shape of 2nd element in pair:', dataset[0][1].shape)

# Create a train and validation subset of variable dataset with torch
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Put train dataset into a loader with 2 batches and put test data in val loader
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True)

# Initialize model, trainer, optimizer and loss function
reg_model = model.get_pretrained_regression_model(100)
trainer = model.Trainer()
optimizer = torch.optim.Adam
loss = torch.nn.MSELoss()
trainer.compile(reg_model, optimizer, learning_rate=0.0001, loss_fn=loss)

trainer.fit(num_epochs = 3, train_loader=train_loader, val_loader=val_loader)


Dataset made up of  20 pairs of data
--------


Epoch 1/3, Function: fit:   0%|          | 0/1 [00:00<?, ?it/s]


TypeError: new(): data must be a sequence (got numpy.float32)